In [129]:
def compareAbsoluteValues(dict1, dict2, key):
    return abs( (dict1.get(key) if dict1.get(key) else 0) - (dict2.get(key) if dict2.get(key) else 0))
def compareValues(dict1, dict2, key):
    return  (dict1.get(key) if dict1.get(key) else 0) - (dict2.get(key) if dict2.get(key) else 0)

In [116]:
orginial = open("theBox.txt").read().split()
new = open("theBox2.txt").read().split()

In [117]:
print(orginial[2])
print(new[1])

th-e
gaston


In [118]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [119]:
import string
table = str.maketrans('', '', string.punctuation)
strippedOrginial = [w.translate(table) for w in orginial]
strippedOrginial = [word.lower() for word in strippedOrginial]
wordfreq = []
for w in strippedOrginial:
    wordfreq.append(strippedOrginial.count(w))


#print("List\n" + str(stripped) + "\n")
#print("Frequencies\n" + str(wordfreq) + "\n")
#print("Pairs\n" + str(list(zip(stripped, wordfreq))))
#pairs = list(zip(strippedOrginial, wordfreq))
pairsOrginial = dict(zip(strippedOrginial, wordfreq))

In [120]:
tableNew = str.maketrans('', '', string.punctuation)
strippedTranscript = [w.translate(tableNew) for w in new]
strippedTranscript = [word.lower() for word in strippedTranscript]
wordfreqTranscript = []
for w in strippedTranscript:
    wordfreqTranscript.append(strippedTranscript.count(w))


#print("List\n" + str(stripped) + "\n")
#print("Frequencies\n" + str(wordfreq) + "\n")
#print("Pairs\n" + str(list(zip(stripped, wordfreq))))
#pairs = list(zip(strippedOrginial, wordfreq))
pairsTranscript = dict(zip(strippedTranscript, wordfreqTranscript))

In [121]:
pairsOrginial['the']

48

In [122]:
countOrginal = 0
countTranscript = 0
for key in pairsOrginial:
    countOrginal = countOrginal + pairsOrginial[key]
print(countOrginal)

for key in pairsTranscript:
    countTranscript = countTranscript + pairsTranscript[key]
print(countTranscript)

553
553


In [123]:
print(pairsTranscript['the'])

48


In [130]:
countDiffFromOrginial = 0
wordsNotConsistentWithOrginial = {}
for key in pairsOrginial:
    countDiffFromOrginial = countDiffFromOrginial + compareAbsoluteValues(pairsOrginial,pairsTranscript,key)
    if pairsOrginial.get(key) != pairsTranscript.get(key):
        wordsNotConsistentWithOrginial[key] = compareValues(pairsOrginial,pairsTranscript,key)
print(wordsNotConsistentWithOrginial)

{'out': 1}


In [131]:
countDiffFromTranscript = 0
wordsNotConsistentWithTranscript = {}
for key in pairsTranscript:
    countDiffFromTranscript = countDiffFromTranscript + compareAbsoluteValues(pairsOrginial,pairsTranscript,key)
    if pairsOrginial.get(key) != pairsTranscript.get(key):
        wordsNotConsistentWithTranscript[key] = compareValues(pairsTranscript,pairsOrginial,key)
print(wordsNotConsistentWithTranscript)

None
1
1
10
9
1
{'gaston': 1, 'out': -1}


In [143]:
def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy

    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint8)
    d = d.reshape((len(r) + 1, len(h) + 1))
    s = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint8)
    s = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    correct = 0
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
                correct = correct +1
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
                s[i][j] = min(substitution, deletion)

    return d[len(r)][len(h)]/(s[len(r)][len(h)]+correct)

In [149]:
print(wer("hey".split(), "hey a".split()))

0.6666666666666666


In [151]:
import sys
import numpy

def editDistance(r, h):
    '''
    This function is to calculate the edit distance of reference sentence and the hypothesis sentence.
    Main algorithm used is dynamic programming.
    Attributes: 
        r -> the list of words produced by splitting reference sentence.
        h -> the list of words produced by splitting hypothesis sentence.
    '''
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8).reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        d[i][0] = i
    for j in range(len(h)+1):
        d[0][j] = j
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitute = d[i-1][j-1] + 1
                insert = d[i][j-1] + 1
                delete = d[i-1][j] + 1
                d[i][j] = min(substitute, insert, delete)
    return d

def getStepList(r, h, d):
    '''
    This function is to get the list of steps in the process of dynamic programming.
    Attributes: 
        r -> the list of words produced by splitting reference sentence.
        h -> the list of words produced by splitting hypothesis sentence.
        d -> the matrix built when calulating the editting distance of h and r.
    '''
    x = len(r)
    y = len(h)
    list = []
    while True:
        if x == 0 and y == 0: 
            break
        elif x >= 1 and y >= 1 and d[x][y] == d[x-1][y-1] and r[x-1] == h[y-1]: 
            list.append("e")
            x = x - 1
            y = y - 1
        elif y >= 1 and d[x][y] == d[x][y-1]+1:
            list.append("i")
            x = x
            y = y - 1
        elif x >= 1 and y >= 1 and d[x][y] == d[x-1][y-1]+1:
            list.append("s")
            x = x - 1
            y = y - 1
        else:
            list.append("d")
            x = x - 1
            y = y
    return list[::-1]

def alignedPrint(list, r, h, result):
    '''
    This funcition is to print the result of comparing reference and hypothesis sentences in an aligned way.
    
    Attributes:
        list   -> the list of steps.
        r      -> the list of words produced by splitting reference sentence.
        h      -> the list of words produced by splitting hypothesis sentence.
        result -> the rate calculated based on edit distance.
    '''
    print("REF:", end=" ")
    for i in range(len(list)):
        if list[i] == "i":
            count = 0
            for j in range(i):
                if list[j] == "d":
                    count += 1
            index = i - count
            print(" "*(len(h[index])), end=" ")
        elif list[i] == "s":
            count1 = 0
            for j in range(i):
                if list[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) < len(h[index2]):
                print(r[index1] + " " * (len(h[index2])-len(r[index1])), end=" ")
            else:
                print(r[index1], end=" "),
        else:
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print(r[index], end=" "),
    print("\nHYP:", end=" ")
    for i in range(len(list)):
        if list[i] == "d":
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print(" " * (len(r[index])), end=" ")
        elif list[i] == "s":
            count1 = 0
            for j in range(i):
                if list[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) > len(h[index2]):
                print(h[index2] + " " * (len(r[index1])-len(h[index2])), end=" ")
            else:
                print(h[index2], end=" ")
        else:
            count = 0
            for j in range(i):
                if list[j] == "d":
                    count += 1
            index = i - count
            print(h[index], end=" ")
    print("\nEVA:", end=" ")
    for i in range(len(list)):
        if list[i] == "d":
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print("D" + " " * (len(r[index])-1), end=" ")
        elif list[i] == "i":
            count = 0
            for j in range(i):
                if list[j] == "d":
                    count += 1
            index = i - count
            print("I" + " " * (len(h[index])-1), end=" ")
        elif list[i] == "s":
            count1 = 0
            for j in range(i):
                if list[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) > len(h[index2]):
                print("S" + " " * (len(r[index1])-1), end=" ")
            else:
                print("S" + " " * (len(h[index2])-1), end=" ")
        else:
            count = 0
            for j in range(i):
                if list[j] == "i":
                    count += 1
            index = i - count
            print(" " * (len(r[index])), end=" ")
    print("\nWER: " + result)

def wer(r, h):
    """
    This is a function that calculate the word error rate in ASR.
    You can use it like this: wer("what is it".split(), "what is".split()) 
    """
    # build the matrix
    d = editDistance(r, h)

    # find out the manipulation steps
    list = getStepList(r, h, d)

    # print the result in aligned way
    result = float(d[len(r)][len(h)]) / len(r) * 100
    result = str("%.2f" % result) + "%"
    alignedPrint(list, r, h, result)



In [ ]:
if __name__ == '__main__':
    filename1 = sys.argv[1]
    filename2 = sys.argv[2]
    with open(filename1, 'r', encoding="utf8") as ref:
        r = ref.read().split()
    with open(filename2, 'r', encoding="utf8") as hyp:
        h = hyp.read().split()
    wer(r, h)  

In [155]:
print(wer('hey you', 'hey you'))

REF: h e y   y o u 
HYP: h e y   y o u 
EVA:               
WER: 0.00%
None
